In [1]:
import ujson
import re
import pandas as pd

from glob import glob
from gensim.models.word2vec import Word2Vec
from itertools import product
from IPython.display import Markdown, display
from wordfreq import top_n_list
from tqdm import tqdm_notebook
from tabulate import tabulate

In [2]:
pd.options.display.max_rows = 1000

In [3]:
def read_tweets(pattern, fcount=None):
    
    paths = glob(pattern)
    
    if fcount:
        paths = paths[:fcount]
    
    tweets = []
    for path in paths:
        with open(path) as fh:
            for line in fh:
                
                tweet = ujson.loads(line)
                text = re.sub('(#|@|http)\S+', '', tweet['body'])
                tokens = re.findall('[a-z0-9]+', text.lower())
                tweets.append(tokens)
                
    return tweets

In [4]:
rural = read_tweets('../../data/geo-lt10k.json/*.json', 60)

In [5]:
urban = read_tweets('../../data/geo-gt1m.json/*.json', 20)

In [6]:
len(rural)

11780936

In [7]:
len(urban)

12325982

In [108]:
rural_model = Word2Vec(rural, min_count=1000, workers=8)

In [109]:
urban_model = Word2Vec(urban, min_count=1000, workers=8)

In [219]:
rural_model.save('../data/models/rural.bin')

In [220]:
urban_model.save('../data/models/urban.bin')

In [110]:
rural_model.most_similar('earth')

[('planet', 0.7846516370773315),
 ('universe', 0.495510071516037),
 ('purpose', 0.4822278320789337),
 ('moon', 0.4742686450481415),
 ('world', 0.4736853539943695),
 ('alien', 0.47179871797561646),
 ('species', 0.46059975028038025),
 ('mars', 0.4551383852958679),
 ('nature', 0.44583749771118164),
 ('shining', 0.4440971910953522)]

In [111]:
urban_model.most_similar('earth')

[('planet', 0.7880303859710693),
 ('darkness', 0.49556586146354675),
 ('nature', 0.48827022314071655),
 ('thrive', 0.4876272678375244),
 ('invisible', 0.48423391580581665),
 ('heaven', 0.4732300639152527),
 ('moon', 0.4650537371635437),
 ('trees', 0.45670124888420105),
 ('species', 0.45475828647613525),
 ('christ', 0.451892614364624)]

In [112]:
def score_shift(m1, m2, token, n=10):
    
    m1_similar = [t for t, _ in m1.most_similar(token, topn=n)]
    m2_similar = [t for t, _ in m2.most_similar(token, topn=n)]
    
    score = 0
    for t1, t2 in product(m1_similar, m2_similar):
        if t1 in m2 and t2 in m2:
            score += m2.similarity(t1, t2)
        
    return score

In [201]:
rural_vocab = set([t for t, v in rural_model.wv.vocab.items() if v.count > 1000])
urban_vocab = set([t for t, v in urban_model.wv.vocab.items() if v.count > 1000])

In [202]:
len(rural_vocab)

7011

In [203]:
len(urban_vocab)

7774

In [204]:
vocab = set.intersection(rural_vocab, urban_vocab)

In [205]:
len(vocab)

6552

In [206]:
data = []

for token in tqdm_notebook(vocab):
    try:
        score = score_shift(urban_model, rural_model, token)
        data.append((token, score))
    except: pass

HBox(children=(IntProgress(value=0, max=6552), HTML(value='')))

In [207]:
df = pd.DataFrame(data, columns=('token', 'score'))

In [208]:
df.sort_values('score').head(300)

,token,score
3566,js,-0.755771
5961,bb,-0.184413
5028,hay,0.000000
4305,ese,0.000000
6318,ye,0.688010
4050,lady,1.000451
3537,lo,1.521036
3523,sc,1.563108
5976,fair,2.445868
4323,dir,3.298084


In [209]:
def compare(token):
    
    u_similar = urban_model.most_similar(token)
    r_similar = rural_model.most_similar(token)
    
    u_tokens = [[t] for t, _ in u_similar]
    r_tokens = [[t] for t, _ in r_similar]
    
    return u_tokens, r_tokens

In [211]:
for token in df.sort_values('score').head(300)['token']:
    u, r = compare(token)
    display(Markdown(f'# {token}'))
    display(Markdown(f'#### Urban'))
    print(tabulate(u))
    display(Markdown(f'#### Rural'))
    print(tabulate(r))

# js

#### Urban

-----------
dev
desktop
user
software
developer
technical
programming
engineer
bot
analytics
-----------


#### Rural

--------
tho
slim
trippin
jk
goofy
atm
cuh
hella
ta
tripping
--------


...

# gt

#### Urban

--------
lt
com
ali
sizes
dot
checkout
mag
coupon
nigerian
cum
--------


#### Rural

--------
lt
com
coupon
promo
scarf
boats
baro
remedies
hacks
necklace
--------
